# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [56]:
#import libraries:
import numpy as np 
from sklearn.preprocessing import normalize 
import cv2
import open3d as o3d

In [57]:
def read_transformations(filename='poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(temp_rot)
    return transformation_list
   
##### Helper function to transform the 3d points to the first camera frame or the ground truth poses. 
def newpoint(dp3, x):
    
    x1 = np.array([
        [x[0][0], x[0][1], x[0][2], x[0][3]],
        [x[1][0], x[1][1], x[1][2], x[1][3]],
        [x[2][0], x[2][1], x[2][2], x[2][3]],
    ])
    pts = np.mat([dp3[0], dp3[1], dp3[2], 1]).T
    mats = np.matmul(x1, pts)
    return mats


## 3d reconstruction from stereo image set. 

#### Code written by Krishna Kumar - 2018122003

 1. We first find the disparity map between the 20 image pairs. We used the CV2 inbuilt function stereoBGM() which generated the disparity map. To get a good disparity map we had  to experiment with the parameters of StereoSGBM. 
 
 2. Then we generated the Q matrix so that along with the disparity matrix. We can reconstruct 3d points corresponding to the image. 
 
    Q=[[1,0,0,-w/2],
       [0,-1,0,h/2],
       [0,0,0,fl],
       [0,0,-bl**(-1),0]]
       
       The basedline and the K matrix were given in calib.txt.  bl - baseline in m.  w,h - width and height of        the image. fl - focal length of the camera. (obtained from K- Matrix). 
 
 3. Then we multiply the Q matrix with the image points agumented with the disparity values (AugMat). So as to recover the 3d point. 
 Each column of the agumented matrix contains contains the [ x y d(x,y) 1] for all the images. 
 
 4. After multiplying the image points are converted to point cloud format for visualization. 
      
 

In [ ]:
#####............Reading the K-matrix and baseline width........#######
calibF=open('calib.txt')
line=0
for lines in calibF:
    if(line==1):
        k= np.array(lines.strip('\n').split(' '))
    if(line==4):
        bDash=lines.split(' ')
        b=float(bDash[0])
    line+=1
kDash=k.astype('float64').reshape(3,3)

worldPoints=[]
pose_sh = []
transformationList=np.array(read_transformations()).astype('float64')
pose0 = np.vstack([transformationList[0], np.array([0, 0, 0, 1])])
pose_sh.append(pose0)
for i in range(1, len(transformationList)):
    posey = np.vstack([transformationList[i], np.array([0, 0, 0, 1])])
    pose_sh.append(posey)


#############................ calcuation of Q matrix ............#####################
    
import matplotlib.pyplot as plt
pathL='img2/'
pathR='img3/'

fl=7.070912e+02
bl = 0.53790448812

xc=185 # Centre pixel
yc=613 # Centre pixel 

# calculation of Q m
f=k[0][0]



# imgLrGB = cv2.cvtColor('/img2/0000000460.png', cv2.COLOR_BGR2RGB)
# grayRrGB = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)


########################.......... Generating the 3d points for stereo imgage pair.......###################

camPoints=[]
colours=np.zeros((370, 1226, 3))
thdoutputs=[]
thdcolours=[]
imgPointMap = {}
imgPointMap_alt = {}
imgPointMap_2 = {}

NumOfPairs=5 # Change this paramter to include more images. 

for i in range(NumOfPairs):
    
    # Reading the images. 
    img2=pathL+'0'*7+str(460+i)+str('.png')
    img3=pathR+'0'*7+str(460+i)+str('.png')
    

    imgLF=cv2.imread(img2)
    imgRF=cv2.imread(img3)
    
    imgL=cv2.cvtColor(imgLF,cv2.COLOR_BGR2RGB)
    imgR=cv2.cvtColor(imgRF,cv2.COLOR_BGR2RGB)
    
    
#########........Use this code to downsample the image if needed...........############

    scale_percent = 100 # percent of original size
    width = int(imgL.shape[1] * scale_percent / 100)
    height = int(imgR.shape[0] * scale_percent / 100)
    dim = (width, height) 
    h, w = imgL.shape[:2]
#     print(h)


###########............. consturction of Q matrix.................##############

    numOfPix= width*height
    Q=np.array([[1,0,0,-w/2],[0,-1,0,h/2],[0,0,0,fl],[0,0,-bl**(-1),0]])
    Q=Q.astype('float64')
    
#     imgL=cv2.resize(imgLF, dim, interpolation = cv2.INTER_jnreconAREA) 
#     imgR=cv2.resize(imgRF, dim, interpolation = cv2.INTER_AREA) 

    window_size = 3 
    
    minDisp = 16
    numDisp = 112 - minDisp
    
###########...Finding disparity map for the stereo pair...########
    
    ImageDisp = cv2.StereoSGBM_create(
        minDisparity=minDisp,             
        blockSize=7,
        numDisparities=numDisp + (2) * 16,
        P1=8*3*window_size**2,    
        P2=32*3*window_size**2,
        uniquenessRatio=12,
        speckleWindowSize=400,
        disp12MaxDiff=1,
        speckleRange=5,
        #preFilterCap=63
    )
    
    
    dispMat = ImageDisp.compute(imgL, imgR).astype(np.float32)
#     print(dispMat)
    
##############...product of Q matix and augumented image points.......#############
### Accumiliating in array thdoutputs and thdcolurs from the image to generated point coud ####

    camPoints = np.zeros((370,1226,3))
    colours = np.zeros((370,1226,3))
    
    imgPointMap[i] = []
    imgPointMap_alt[i] = []
    imgPointMap_2[i] = []
    for k in range(len(imgL)):
        for j in range(len(imgL[k])):
            colours[k][j] = imgL[k][j]
            pt_temp = Q @ np.array([k, j, dispMat[k][j]/16.0, 1]).T
            W = pt_temp[3]
            to_ap = [pt_temp[0]/W, pt_temp[1]/W, pt_temp[2]/W]
            camPoints[k][j] = to_ap
            imgPointMap[i].append(([k, j], to_ap))
            if dispMat[i][j] > dispMat.min():
                imgPointMap_alt[i].append(([k, j], to_ap))
                # This step takes the above project 3d, and transforms them according to the ground truth poses.
                pt_new_temp = np.array(newpoint(to_ap, pose_sh[i]).T).astype('float32')[0]
                pt_add = [pt_new_temp[0], pt_new_temp[1], pt_new_temp[2]]
                imgPointMap_2[i].append(([k, j], pt_add))




    # applying mask for better results. And appending the 3d points for all the image pairs. 
    mask_map = dispMat > dispMat.min()
    o_camPoints = camPoints[mask_map]
    o_colours = colours[mask_map]
    o_colours =(o_colours/255).astype('float64')
    for j,pts in enumerate(o_camPoints):
        thdoutputs.append(np.array(newpoint(pts, pose_sh[i]).T).astype('float32')[0])
        thdcolours.append(o_colours[j])
        
thdoutputs = np.array(thdoutputs)
thdcolours = np.array(thdcolours)

#print(thdoutputs)
# print(thdcolours.shape)

In [ ]:
################............. Visualizing the point cloud ...................#####################
accPntCld=[]
pntCld= o3d.geometry.PointCloud()
pntCld.points= o3d.utility.Vector3dVector(thdoutputs)
pntCld.colors= o3d.utility.Vector3dVector(thdcolours)
o3d.io.write_point_cloud("all_pcs.ply", pntCld)
o3d.visualization.draw_geometries([pntCld])

----
### PART 2: Motion estimation using iterative PnP

###### Coded by Pulkit Gera (Roll no: 20171035)

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

In [ ]:
# Correct R|t Matrix 
Q = np.array([ [-9.1e-01, 5.5e-02, -4.2e-01, -1.9e+02],
               [4.2e-02, 9.983072e-01, 4.2e-02, 1.7e+00],
               [4.2e-01, 2.1e-02, -9.2e-01, 5.5e+01],
                 [0,0,0,1]])
Q=np.linalg.inv(Q)[:-1]

#Calib Matrix 
K = kDash

# Correct projection matrix, the one we should be getting after GN 
P = K@Q # 3x4 matrix 
print("Correct projection matrix: \n", P)

In [ ]:
points = np.random.randint(low=0,high=len(imgPointMap_alt[0]),size=100)
pts_3d = []
gt_pts_2d = []
pts_3d = [imgPointMap_alt[0][i][1] for i in range(0,10000,100)]
pts_3d = np.asarray(pts_3d)
pts_3d = np.hstack((pts_3d,np.ones((100,1))))
gt_pts_2d = (P@(pts_3d.T)).T
for i in range(100):
    gt_pts_2d[i] /= gt_pts_2d[i,-1]
print(gt_pts_2d.shape,pts_3d.shape)

## Gauss-Newton 
- Now we run the gauss newton algorithm and try to refine P with every iteration. 
- We take an initial estimate of P that is close to the actual value of P, because otherwise, GN would get stuck in local minima. 

In [ ]:
P_est = np.array([ [-8.8e+02,5.4e+01,-2.4e+02,-1.5e+05],
                   [-3.6e+01,7.1e+02,-1.4e+02,1.4e+02],
                   [-4.1e-01,4.1e-02,-8.9e-01,-2.9e+01] ])

- Now running Gauss Newton algorithm. With every iteration, we try to find a better estimate for P
- Please refer to the handwritten notes attached along with submission about Jacobian
- Every iteration, we do P + delP, calculated from jacobian

In [ ]:
print("Running Gauss newton: \n ")
while True: 
    pts_2d = [] # corresponding 2D location estimated based on our P matrix 
    J = []
    curr_pts_2d = (P_est@(pts_3d.T)).T
    for i in range(100):
        curr_pts_2d[i] /= curr_pts_2d[i,-1]
    residual = gt_pts_2d-curr_pts_2d
    residual = residual[:,:-1].flatten()
    objective = residual.T@residual
     
    print("squared sum error:", objective)
    if(objective < 0.0001):
        break
        
    for i in range(pts_3d.shape[0]):
        X_num,Y_num,Z_denom = P_est@pts_3d[i]
        X_pt,Y_pt,Z_pt,_ = pts_3d[i]
        
        J.append(np.array([X_pt/Z_denom,Y_pt/Z_denom,Z_pt/Z_denom,1/Z_denom,0,0,0,0,
                             (-X_num*X_pt)/(Z_denom**2),(-X_num*Y_pt)/(Z_denom**2),(-X_num*Z_pt)/(Z_denom**2),-1/(Z_denom**2)]))
        J.append(np.array([0,0,0,0,X_pt/Z_denom,Y_pt/Z_denom,Z_pt/Z_denom,1/Z_denom,
                             (-Y_num*X_pt)/(Z_denom**2),(-Y_num*Y_pt)/(Z_denom**2),(-Y_num*Z_pt)/(Z_denom**2),-1/(Z_denom**2)]))
    J = np.asarray(J).reshape(200,12)
    H = J.T@J
    L = np.linalg.pinv(H)
    update = J.T@residual
    delP = (L@update).reshape(3,4)
    P_est = P_est+delP

print("Predicted projection matrix after Gauss Newton: \n")
print(P_est)

        

We see that the algorithm converges after a few steps. This is the case for all the images we are running it on.

1. We already have the 3D points generated from the previous part, which we used to render the point cloud. We, thereafter, find the 2D pixel points that correspond to these 3D points. We do so by multiplying each 3D point by the projection matrix, obtained from R, K, t. K has been given to us and R|t we get from the poses.txt.


2. We take inverse of R|t matrix because we want to go from world to camera. Hence, we add a new row to make it 4X4. Basically, P = KRt


3. After this is done, we find the actual 2D points that correspond to the 3D point by multiplying each 3D point with P.


4. Once this is complete, we run the Gauss Newton algo and try to refine P with every iteration. We take an intial estimate of P that is close to the actual value of P, because otherwise, we shall be stuck around a local minima. 


5. Now, running Gauss Newton algo, with every iteration, we try to find a better estimate for P. Every iteration, we do a P + delP calculated from the Jacobian

### Part-B Project -2 

##### Answered by Pulkit Gera (20171035) and Krishna Kumar together by discussion. 


1. Basics - How do we know this (camera_ind) information in practical setting? Explain.

Answer - In the Structre for motion (SFM) pipeline, In the front end, intially mutiple images of an object are being taken . From these multiple images, common features are detected from consequtive image sets (using methods like SIFT or ORB and so on). After we have detected these feature points, the fundamental matrix of the image points are detected. (Using algorithms like the eight point algorithm). From which we can find the information between the points like Rotation and translation matrices. Once we know the Rotation and translation matrices. We can find out reconstruct the 3d points world points by considering by using methods like pnp. 

2. Basics - How do we know this (point_ind) information in practical setting? Explain.

Indexing to 2D points and labelling to 3D points helps in tracking which 3D points generated by 2D points. Similar is answered for previous question. Initial estimates for 3D points is done by triagnulation and backprojection of 2D points.

3. Transformations - project() function: In the project() function, would it make any difference if I do translate first, then rotate? Why/why not?

Project function is basically roatating from the world frame to local frame and translating i.e shift the origin with local camara frame. This projection is required to transform 3D points projected in aribitary world frame to local carama frame. The translation or shifting vector is in local camara frame where as the projected 3D points are in different frame. It is not possible to use the translation vector of different frames. Hence, it is required to bring the projected 3D points first to local camara frame by rotation to ensure that rotated projection and translation vector are in same frame. 
update answer 3 with However, in this case it does not, and the output is **exactly the same* regardless of the order , this happens because the translation is along the direction of the rotation axis in the project function and thus the result is same regardless of order.

4. m above is not "MN" (2) unlike our lecture notes. Why is that so? (bundle adjustment sparsity function). 

Ans: loss function measures the deviation between observed 2D points and reprojected 2D points. 2D points contribtutes to two resideual points as two cooridnates. Hence, jacobian here contributes two residuals, unlike in the class. In class it is assumed that, M images contributes N points. So, M*N 2D points were considered.

5. Why are we doing n_cameras * 9 here instead of n_cameras * 12? Recollect :Every i. individual motion Jacobian was (1)12 in our lecture notes. 

Ans: This number represents the number of camara parameters considered for optmization to estimate the 3D world points. Unlike in class, here we considered only 9 camara parameters viz., 3 for translation, 3 for rotation, 2 for distortion and one for focal length. Hence n_camaras9 instead of n_camaras*12. 

6. Explain what you understand from above 6 lines of code by coding? (bundle adjustemnt sparsity function). 

Ans: First for loop sets the 2 lines of 9 colums of jacobian matrix correpsonds to 9 camara parameters set to 1. This is done every two rows at a time. In a similar way, second for loop sets three colums of world frame of ith observation sets to one. Rest of the columns of other observation are left to zero. This way required sparsity is achieved.